In [1]:
# !pip install transformers

In [2]:
import os, gc, sys, time, collections, random
import numpy as np
import pandas as pd

from typing import Dict, Optional, Union, Any, List, Tuple

from pathlib import Path
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn

import torch.utils.data as D
from torch.utils.data.dataset import Dataset, IterableDataset
from torch.utils.data.dataloader import DataLoader

from transformers import Trainer
from transformers import AutoModel, AutoTokenizer
from transformers.data.processors.utils import InputFeatures
from transformers import TrainingArguments
from transformers.trainer_utils import EvalLoopOutput
from transformers.trainer import logging
from transformers.file_utils import is_torch_tpu_available, is_sagemaker_mp_enabled
from transformers.trainer_pt_utils import find_batch_size, nested_concat, nested_numpify, nested_truncate, nested_detach
from transformers import EarlyStoppingCallback

### Folders and Dataframes

In [3]:
DATA_PATH = Path('/home/commonlit/data/')
assert DATA_PATH.exists()
MODELS_PATH = Path('/home/commonlit/models/')
if not MODELS_PATH.exists():
    os.mkdir(MODELS_PATH)
assert MODELS_PATH.exists()

In [4]:
!ls {DATA_PATH}

commonlitreadabilityprize.zip  sample_submission.csv  test.csv	train.csv


In [5]:
train_df = pd.read_csv(DATA_PATH/'train.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
sample_df = pd.read_csv(DATA_PATH/'sample_submission.csv')

In [6]:
train_df

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845
...,...,...,...,...,...,...
2829,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900
2830,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648
2831,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866
2832,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128


### Prepare Cross Validation

In [7]:
target = train_df['target'].to_numpy()

In [8]:
num_bins = int(np.floor(np.log2(len(train_df))) + 1)
train_df.loc[:, 'bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)

In [9]:
train_df[['target', 'bins']].groupby(['bins']).agg(['mean', 'count'])

target      
          mean count
bins                
0    -3.413097    43
1    -2.969369    79
2    -2.526589   172
3    -2.106393   269
4    -1.652726   366
5    -1.201502   418
6    -0.748738   481
7    -0.309800   405
8     0.130016   312
9     0.560407   183
10    0.978923    83
11    1.399764    23

In [10]:
kf = StratifiedKFold(n_splits=num_bins)

In [11]:
for i, (t_, v_) in enumerate(kf.split(X=train_df, y=train_df.bins.values)):
    train_df.loc[v_, 'kfold'] = i

In [12]:
train_df['kfold'] = train_df['kfold'].astype(np.uint8)

In [13]:
train_df = train_df.drop('bins', axis=1)

In [14]:
train_df

,id,url_legal,license,excerpt,target,standard_error,kfold
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,0
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,0
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,0
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,0
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,0
...,...,...,...,...,...,...,...
2829,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900,11
2830,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648,11
2831,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866,11
2832,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128,11


### Metrics

In [15]:
def rmse_score(y_true, y_pred):
    return np.sqrt(np.mean((y_pred - y_true) ** 2))

def rmse_score_2(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [16]:
a = np.random.rand(10)
b = np.random.rand(10)

In [17]:
rmse_score(a, b), rmse_score_2(a, b)

(0.47020624681904855, 0.47020624681904855)

### Configuration

In [18]:
class CONFIG():
    model_name = 'distilbert'
    batch_size = 128
    max_len = 256
    save_dir = f'trained/{model_name}'
    num_workers = 2
    epochs = 20
    pretrained_transformers_model = f'{model_name}-base-uncased'

In [19]:
cfg = CONFIG()

### Prepare train test split

In [20]:
def create_split(fold = [1]):
    valid_df = train_df[train_df['kfold'].isin(fold)]
    valid_text = valid_df['excerpt'].values
    valid_target = valid_df['target'].values
    training_df = train_df[~train_df['kfold'].isin(fold)]
    train_text = training_df['excerpt'].values
    train_target = training_df['target'].values
    return train_text, train_target, valid_text, valid_target

In [21]:
train_text, train_target, valid_text, valid_target = create_split([1])
len(train_text), len(valid_text)

(2597, 237)

### Prepare Tokenizers

In [22]:
tokenizer = AutoTokenizer.from_pretrained(cfg.pretrained_transformers_model)
# Save the tokenizer so that you can download the files and move it to a Kaggle dataset.
tokenizer.save_pretrained(cfg.save_dir)

('trained/distilbert/tokenizer_config.json',
 'trained/distilbert/special_tokens_map.json',
 'trained/distilbert/vocab.txt',
 'trained/distilbert/added_tokens.json',
 'trained/distilbert/tokenizer.json')

In [23]:
encoded_dict = tokenizer(train_df['excerpt'].values[0],
                                return_tensors='pt',
                                max_length=cfg.max_len,
                                padding='max_length',
                                truncation=True)
decoded = tokenizer.decode(encoded_dict["input_ids"].squeeze())
decoded

'[CLS] when the young people returned to the ballroom, it presented a decidedly changed appearance. instead of an interior scene, it was a winter landscape. the floor was covered with snow - white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. the numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches. at each end of the room, on the wall, hung a beautiful bear - skin rug. these rugs were for prizes, one for the girls and one for the boys. and this was the game. the girls were gathered at one end of the room and the boys at the other, and one end was called the north pole, and the other the south pole. each player was given a small flag which they were to plant on reaching the pole. this would have been an easy matter, but each traveller was obliged to wear snowsho

In [24]:
encoded_dict['input_ids'].shape

torch.Size([1, 256])

In [25]:
def convert_to_list(t):
    return t.flatten().long()

class CommonLitDataset(nn.Module):
    def __init__(self, text, target, tokenizer, max_len=128):
        self.excerpt = text
        self.target = target
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        return InputFeatures(input_ids=convert_to_list(encode['input_ids']),
                      attention_mask=convert_to_list(encode['attention_mask']),
                      label=torch.tensor(self.target[idx]))
    
    def __len__(self):
        return len(self.excerpt)

In [26]:
def create_train_valid_ds(tokenizer, train_text, train_target, valid_text, valid_target):
    train_ds = CommonLitDataset(train_text, train_target, tokenizer, cfg.max_len)
    valid_ds = CommonLitDataset(valid_text, valid_target, tokenizer, cfg.max_len)
    return train_ds, valid_ds

In [27]:
# train_dl = D.DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True, num_workers=cfg.num_workers)
# train_dl = D.DataLoader(valid_ds, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)

In [28]:
# encode, target = next(iter(train_dl))

In [29]:
# encode.keys(), target.shape, encode['input_ids'].shape, encode['attention_mask'].shape

In [30]:
# encode['input_ids'][0].squeeze()

### Model

In [31]:
# You can use a Transformer model of your choice.
# transformer_model = DistilBertModel.from_pretrained(cfg.pretrained_transformers_model)
transformer_model = AutoModel.from_pretrained(cfg.pretrained_transformers_model)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
# transformer_out = transformer_model(input_ids=encode['input_ids'].squeeze(), attention_mask=encode['attention_mask'].squeeze())

In [33]:
# dict(transformer_out)['last_hidden_state'].shape

In [34]:
# torch.mean(transformer_out.last_hidden_state, axis=1).shape

In [35]:
# sample_layer = nn.Linear(768, 1)

In [36]:
# sample_layer(torch.mean(transformer_out.last_hidden_state, axis=1)).shape

In [37]:
from transformers import PreTrainedModel

class CommonLitModel(PreTrainedModel):
    def __init__(self):
        super(PreTrainedModel, self).__init__()
        self.transformer_model = AutoModel.from_pretrained(cfg.pretrained_transformers_model)
        self.drop = nn.Dropout(0.5)
        self.out = nn.Linear(768, 1)
        self.config = self.transformer_model.config
        
    def forward(self, input_ids, attention_mask):
        transformer_out = self.transformer_model(input_ids=input_ids.squeeze(), attention_mask=attention_mask.squeeze(), output_hidden_states=False)
        x = transformer_out.last_hidden_state[:, 0, :]
        x = self.drop(x)
        x = self.out(x)
        return x
    
    def floating_point_ops(self, inputs: Dict[str, Union[torch.Tensor, Any]]):
        """
        For models that inherit from :class:`~transformers.PreTrainedModel`, uses that method to compute the number of
        floating point operations for every backward + forward pass. If using another model, either implement such a
        method in the model or subclass and override this method.
        Args:
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.
        Returns:
            :obj:`int`: The number of floating-point operations.
        """
        return 0

In [38]:
model = CommonLitModel()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [39]:
encoded_dict.input_ids.shape

torch.Size([1, 256])

In [40]:
transformer_model = transformer_model.cuda()
sample_out = transformer_model(encoded_dict.input_ids.cuda(), encoded_dict.attention_mask.cuda(), output_hidden_states=True)

In [41]:
train_ds, valid_ds = create_train_valid_ds(tokenizer, train_text, train_target, valid_text, valid_target)

In [42]:
encode = train_ds[0]

In [43]:
encode.attention_mask.unsqueeze(0).shape, encoded_dict.input_ids.shape

(torch.Size([1, 256]), torch.Size([1, 256]))

In [44]:
sample_out = transformer_model(encode.input_ids.unsqueeze(0).cuda(), encode.attention_mask.cuda())

In [45]:
sample_out

BaseModelOutput(last_hidden_state=tensor([[[-0.3542, -0.0775,  0.0143,  ...,  0.0467,  0.3917,  0.5215],
         [-0.7111, -0.0619,  0.5459,  ..., -0.0886,  0.1610,  0.1668],
         [-0.2945, -0.3932,  0.3832,  ...,  0.0047,  0.3847, -0.2494],
         ...,
         [-0.0588, -0.1648,  0.2311,  ...,  0.2598,  0.0082, -0.2071],
         [ 0.3317, -0.1402, -0.0694,  ...,  0.4430, -0.0540, -0.2125],
         [ 0.2887, -0.0008, -0.0171,  ...,  0.2667,  0.1959, -0.1965]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward>), hidden_states=None, attentions=None)

### Training

In [46]:
loss_fct = nn.MSELoss()

In [47]:
logger = logging.get_logger(__name__)

class CommonLitTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        input_ids = inputs.pop("input_ids")
        attention_mask = inputs.pop("attention_mask")
        outputs = model(input_ids, attention_mask)
        logits = outputs
        loss = loss_fct(logits.flatten(),
                        labels.float().flatten())
        zero_cat = torch.zeros([1, 1]).to(outputs.device)
        return (loss, torch.cat([zero_cat, outputs])) if return_outputs else loss

In [48]:
def create_training_args(fold):
    training_args = TrainingArguments(
        output_dir=str(MODELS_PATH/f'{cfg.model_name}-{fold}'),
        overwrite_output_dir=True,
        do_train=True,
        do_eval=True,
        per_device_train_batch_size=cfg.batch_size,
        per_device_eval_batch_size=cfg.batch_size,
        num_train_epochs=cfg.epochs,
        logging_strategy="epoch",
        logging_first_step=True,
        save_steps=40000,
        fp16=True,
        evaluation_strategy="epoch",
        save_total_limit = 2,
        load_best_model_at_end=True,
        metric_for_best_model='mse',
        greater_is_better=False
    )
    return training_args

In [49]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    return {'mse': mean_squared_error(logits, labels), 'rmse': rmse_score_2(logits, labels)}

In [50]:
bin_list = list(range(num_bins))
random.shuffle(bin_list)
bin_list

[2, 1, 10, 6, 7, 9, 4, 0, 11, 3, 5, 8]

In [51]:
!rm -rf /home/commonlit/models/distilbert-*

In [52]:
%%time

bin_step = 1
bestmodels = []
for i in range(0, num_bins, bin_step):
    train_bins = bin_list[i:i+bin_step]
    print('train_bins', f'{i}: {train_bins}')
    tokenizer = AutoTokenizer.from_pretrained(cfg.pretrained_transformers_model)
    train_text, train_target, valid_text, valid_target = create_split([i])
    train_ds, valid_ds = create_train_valid_ds(tokenizer, train_text, train_target, valid_text, valid_target)
    training_args = create_training_args(i)
    model = CommonLitModel()
    trainer = CommonLitTrainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=valid_ds,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=9)]
    )
    trainer.train()
    trainer.save_model()
    print('training_args.output_dir', training_args.output_dir)
    tokenizer.save_pretrained(training_args.output_dir)
    result = trainer.evaluate()
    bestmodels.append(trainer.state.best_model_checkpoint)
    print('best_model_checkpoint', trainer.state.best_model_checkpoint)
    print('result', result)

train_bins 0: [2]


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch,Training Loss,Validation Loss,Mse,Rmse
1,0.953300,0.475219,0.475218,0.689361
2,0.345000,0.312149,0.312149,0.558703
3,0.269400,0.369456,0.369456,0.607829
4,0.178100,0.498848,0.498848,0.706292
5,0.118600,0.312969,0.312969,0.559436
6,0.079700,0.393527,0.393527,0.627318
7,0.065000,0.376406,0.376406,0.613519
8,0.046200,0.436803,0.436803,0.660911
9,0.037600,0.405220,0.405220,0.636569
10,0.032400,0.336844,0.336844,0.580383


training_args.output_dir /home/commonlit/models/distilbert-0


best_model_checkpoint /home/commonlit/models/distilbert-0/checkpoint-42
result {'eval_loss': 0.3121494948863983, 'eval_mse': 0.3121494948863983, 'eval_rmse': 0.5587034225463867, 'eval_runtime': 0.7572, 'eval_samples_per_second': 312.99, 'epoch': 11.0, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 1510507008}
train_bins 1: [1]


...

best_model_checkpoint /home/commonlit/models/distilbert-10/checkpoint-105
result {'eval_loss': 0.2995011806488037, 'eval_mse': 0.2995011508464813, 'eval_rmse': 0.547266960144043, 'eval_runtime': 0.7535, 'eval_samples_per_second': 313.205, 'epoch': 14.0, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 1510507008}
train_bins 11: [8]


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch,Training Loss,Validation Loss,Mse,Rmse
1,1.008400,0.411417,0.411417,0.641418
2,0.362400,0.375421,0.375421,0.612716
3,0.266400,0.325758,0.325758,0.570752
4,0.194600,0.320860,0.320860,0.566445
5,0.131300,0.379985,0.379985,0.616430
6,0.100500,0.321108,0.321108,0.566664
7,0.082400,0.442540,0.442540,0.665236
8,0.069700,0.345488,0.345488,0.587782
9,0.058000,0.434074,0.434074,0.658843
10,0.047900,0.417398,0.417398,0.646063


training_args.output_dir /home/commonlit/models/distilbert-11


best_model_checkpoint /home/commonlit/models/distilbert-11/checkpoint-84
result {'eval_loss': 0.3208598494529724, 'eval_mse': 0.3208598792552948, 'eval_rmse': 0.5664449334144592, 'eval_runtime': 0.7558, 'eval_samples_per_second': 312.266, 'epoch': 13.0, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 1510507008}
CPU times: user 57min 44s, sys: 13min 34s, total: 1h 11min 19s
Wall time: 40min 49s


In [53]:
BEST_MODEL_FOLDER = MODELS_PATH/cfg.model_name/'best'
!rm -rf {BEST_MODEL_FOLDER}
!mkdir -p {BEST_MODEL_FOLDER}

In [54]:
from shutil import copyfile

for i, best_model in enumerate(bestmodels):
    best_model_file = f'{best_model}/pytorch_model.bin'
    copyfile(best_model_file, f'{BEST_MODEL_FOLDER}/{i}_pytorch_model.bin')
    tokenizer_path = Path(BEST_MODEL_FOLDER/f'tokenizer-{i}')
    tokenizer_path.mkdir(parents=True, exist_ok=True)
    assert tokenizer_path.exists()
    
    tokenizer_json = Path(f'{MODELS_PATH/cfg.model_name}-{i}/tokenizer.json')
    assert tokenizer_json.exists()
    copyfile(tokenizer_json, tokenizer_path/'tokenizer.json')
    
    vocab_txt = Path(f'{MODELS_PATH/cfg.model_name}-{i}/vocab.txt')
    assert vocab_txt.exists()
    copyfile(vocab_txt, tokenizer_path/'vocab.txt')
    
    config_json = Path(f'{MODELS_PATH/cfg.model_name}-{i}/config.json')
    assert config_json.exists()
    copyfile(config_json, tokenizer_path/'config.json')

In [55]:
import shutil

shutil.make_archive(MODELS_PATH/cfg.model_name/'best_models', 'zip', BEST_MODEL_FOLDER)

'/home/commonlit/models/distilbert/best_models.zip'